# DataCleaning Notebook - AAAG

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [2]:
df_train = pd.read_csv('train.csv',index_col ="Id")
df_train.shape

(1460, 80)

In [3]:
df_train.SalePrice = np.log1p(df_train.SalePrice)
df_train.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
442,90,RL,92.0,12108,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,Duplex,1Story,4,4,1955,1955,Gable,CompShg,VinylSd,VinylSd,BrkFace,270.0,TA,TA,CBlock,TA,TA,No,ALQ,133,Unf,0,1307,1440,GasA,TA,N,FuseF,1440,0,0,1440,0,0,2,0,4,2,Fa,8,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,11.678448
389,20,RL,93.0,9382,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,125.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,1468,1468,GasA,Ex,Y,SBrkr,1479,0,0,1479,0,0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,1999.0,RFn,2,577,TA,TA,Y,120,25,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal,12.160034
559,60,RL,57.0,21872,Pave,NaN,IR2,HLS,AllPub,FR2,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,PConc,Gd,TA,Gd,GLQ,604,Unf,0,125,729,GasA,Ex,Y,SBrkr,729,717,0,1446,0,1,2,1,3,1,TA,6,Typ,1,TA,Attchd,1996.0,Unf,2,406,TA,TA,Y,264,22,0,0,0,0,NaN,NaN,NaN,0,8,2008,WD,Normal,12.072547
771,85,RL,NaN,7252,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,Norm,Norm,1Fam,SFoyer,5,5,1982,1982,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,Gd,TA,Av,GLQ,685,Unf,0,173,858,GasA,TA,Y,SBrkr,858,0,0,858,1,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1983.0,Unf,2,576,TA,TA,Y,120,0,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal,11.812296
874,40,RL,60.0,12144,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1949,1950,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,CBlock,TA,TA,No,Rec,375,Unf,0,457,832,GasA,Gd,Y,SBrkr,1036,0,232,1268,0,0,1,0,3,1,TA,6,Typ,1,Gd,Attchd,1949.0,Unf,1,288,TA,TA,Y,0,28,0,0,0,0,NaN,NaN,Othr,0,9,2009,WD,Normal,11.798112
267,60,RL,70.0,11207,Pave,NaN,IR1,HLS,AllPub,FR2,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1997,1997,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,714,Unf,0,88,802,GasA,Gd,Y,SBrkr,802,709,0,1511,1,0,2,1,3,1,TA,8,Typ,1,TA,Attchd,1997.0,Fin,2,413,TA,TA,Y,95,75,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,12.128117
234,20,RL,75.0,10650,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,1Fam,1Story,5,6,1976,1976,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,Gd,Av,LwQ,182,ALQ,712,0,894,GasA,TA,Y,SBrkr,894,0,0,894,1,0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1976.0,Unf,1,308,TA,TA,Y,365,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,11.761355
165,40,RM,40.0,5400,Pave,Pave,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1Story,6,7,1926,2004,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,BrkTil,TA,TA,Mn,LwQ,370,Unf,0,779,1149,GasA,Gd,Y,FuseA,1149,467,0,1616,0,0,2,0,3,1,Gd,5,Typ,0,NaN,Detchd,1926.0,Unf,1,216,TA,TA,Y,0,0,183,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Normal,11.931642
1207,20,RH,NaN,8900,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1Story,4,4,1966,1966,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,1056,Unf,0,0,1056,GasA,TA,Y,SBrkr,1056,0,0,1056,1,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1966.0,Unf,1,384,TA,TA,Y,0,42,0,0,0,0,NaN,MnPrv,NaN,0,11,2006,WD,Normal,11.580593


In [4]:
#define function to dummify continuous porch, deck and pool variables
def make_binary(X,vrs):
    '''
    This is a mutating function that replaces any nonzero value with 1
    ----------
    X: dataframe
    vrs: list of variables or single variable
    '''
    for var in vrs:
        df_train.loc[df_train[var]>0,var]=1

In [5]:
def my_impute(df_col,method='replace',replace_with=None,missing_string=None,convert_to_numeric='infer'):
    '''
    - This function takes works for the columns mentioned above. 
    - df_col: take a column series 
    - replace: replace a null with a string defined by replace_with
    - replace_with: the string that replaces the null value
    - missing_string: define what constitutes a null value which will be replaced
    - convert_to_numeric: takes in "infer", "force", "no". "Infer" automatically senses what is 
    - the best action. Converts variable type to numeric. 
    '''
    new_series=df_col.copy()
    try:
        if (df_col.str.contains('Shed').any() and df_col.str.contains('TenC').any()):
            misc_dict={'Gar2':'Shed',
                      'Othr':'No Shed',
                      'TenC':'No Shed'}
            new_series.replace(misc_dict,inplace=True)
        if missing_string!=None:
            missing_dict={null_word:None for i in missing_string}
            new_series.replace(missing_dict,inplace=True) 
        if method=='replace':
            if replace_with==None:
                raise ValueError('Must provide replacement string!')
            new_series[new_series.isnull()]=replace_with
        elif method=='mode':
            new_series[new_series.isnull()]=new_series.mode(dropna=True)[0]
        if convert_to_numeric=='force': 
            score_dict={'TA':3,
                       'Gd':4,
                       'Ex':5,
                       'Fa':2,
                       'Po':1,
                       replace_with:0}
            new_series.replace(score_dict,inplace=True)
            return new_series
        elif convert_to_numeric=='infer':
            if new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^TA$)',regex=True).any() and \
                new_series.str.contains('(^Fa$)',regex=True).any():
                score_dict={'TA':3,
                           'Gd':4,
                           'Ex':5,
                           'Fa':2,
                           'Po':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            elif new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^Av$)',regex=True).any() and \
                new_series.str.contains('(^Mn$)',regex=True).any() and \
                new_series.str.contains('(^No$)',regex=True).any():
                score_dict={'Gd':4,
                           'Av':3,
                           'Mn':2,
                           'No':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            else:
                return new_series
        elif convert_to_numeric=='no':
            return new_series
        else:
            return new_series
    except:
        try: 
            if method=='mean':
                new_series[new_series.isnull()]=new_series.mean(skipna=True)
                return new_series
            elif method=='median':
                new_series[new_series.isnull()]=new_series.median(skipna=True)[0]
                return new_series
        except Exception as e:
            print(type(e),e)

In [6]:
# Clean:          
# - PavedDrive
# - SaleCondition
# - SaleType
# - Street
# - Central Air
# - Functional
# - Heating
# - 1stFlrSF
# - 2ndFlrSF
# - GrLivArea
# - LotArea
# - MSZone
# - OverallCond
# - OverallQual
# - TotalBsmtSF
# - YearBuilt
# - YearRemodAdd

# Drop
# - PoolQC
# - BsmtFinType2
# - Utilities
# - BsmtFinSF1  
# - BsmtFinSF2  
# - BsmtUnSF 
# - LowQualFinSF
# - KitchenAbvGr 
# - MiscVal 
df_train = df_train.drop(['PoolQC','BsmtFinType2','Utilities',\
                          'BsmtFinSF1', 'BsmtFinSF2',\
                          'BsmtUnfSF','LowQualFinSF','KitchenAbvGr', 'MiscVal'], axis=1)

# pply the make binary function to the porch, deck, and pool variables (MUTATING!)
vrs = ['3SsnPorch', 'EnclosedPorch', 'OpenPorchSF','PoolArea','ScreenPorch', 'WoodDeckSF']
make_binary(df_train,vrs)

# Converting to category/string
# - MSSubClass
# - MoSold
df_train['MSSubClass'] = df_train['MSSubClass'].astype('category')
df_train['MoSold'] = df_train['MoSold'].astype('category')

# Impute with mode/median
# - Electrical 
df_train.loc[:,'Electrical']=my_impute(df_train.loc[:,'Electrical'],method='mode',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with mean 
# - GarageYrBlt
df_train.loc[:,'GarageYrBlt']=my_impute(df_train.loc[:,'GarageYrBlt'],method='mean',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with groupedby median of neighborhood
# - LotFrontage
df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Replace with N/A with “no fence”, "No Alley" "No Basement" etc.
# - Fence
# - Alley
# - GarageFinish
# - GarageType
# - BsmtFinType1
df_train.loc[:,'Fence']=my_impute(df_train.loc[:,'Fence'],method='replace',replace_with='No Fence',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'Alley']=my_impute(df_train.loc[:,'Alley'],method='replace',replace_with='No Alley',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageFinish']=my_impute(df_train.loc[:,'GarageFinish'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageType']=my_impute(df_train.loc[:,'GarageType'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtFinType1']=my_impute(df_train.loc[:,'BsmtFinType1'],method='replace',replace_with='No Basement',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'MasVnrType']=my_impute(df_train.loc[:,'MasVnrType'],method='replace',replace_with='No Masonry Veneer',missing_string=None,convert_to_numeric='no')

# Replace with N/A with 0 and turning ordinal into continuous variable
# - BsmtCond
# - BsmtExposure
# - BsmtQual
# - GarageCond
# - GarageQual
# - FireplaceQu
# - HeatingQC
# - KitchenQual
df_train.loc[:,'BsmtCond']=my_impute(df_train.loc[:,'BsmtCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtExposure']=my_impute(df_train.loc[:,'BsmtExposure'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtQual']=my_impute(df_train.loc[:,'BsmtQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageCond']=my_impute(df_train.loc[:,'GarageCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageQual']=my_impute(df_train.loc[:,'GarageQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'FireplaceQu']=my_impute(df_train.loc[:,'FireplaceQu'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'HeatingQC']=my_impute(df_train.loc[:,'HeatingQC'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'KitchenQual']=my_impute(df_train.loc[:,'KitchenQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterCond']=my_impute(df_train.loc[:,'ExterCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterQual']=my_impute(df_train.loc[:,'ExterQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')

# Replace with N/A with 0 only
# - MasVnrArea
# - GarageArea
# - GarageCars
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(0)
df_train['GarageArea'] = df_train['GarageArea'].fillna(0)
df_train['GarageCars'] = df_train['GarageCars'].fillna(0)

# Lumping features into shed and no shed
# - MiscFeature
df_train.loc[:,'MiscFeature']=my_impute(df_train.loc[:,'MiscFeature'],method='replace',replace_with='No Shed',missing_string=None,convert_to_numeric='infer')

# Check: 
df_train.isnull().sum().sum()

/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0

In [7]:
df_train.shape

(1460, 71)

In [8]:
df_train.to_csv('Version_5_df.csv')